In [1]:
%load_ext autoreload
%autoreload 2
import os
import shutil
from typing import Any, Dict, List

from langchain.schema import BaseMemory
from pydantic import BaseModel
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI, LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.agents.agent_toolkits import VectorStoreMemoryToolkit

In [4]:
dataset_path = "./vectorstore/"
os.makedirs(dataset_path, exist_ok=True)
vectorstore = DeepLake(
    dataset_path=dataset_path,
    embedding_function=OpenAIEmbeddings()
)

vectorstore_toolkit = VectorStoreMemoryToolkit(vectorstore=vectorstore, num_results=1)

In [5]:
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=1)
tools = vectorstore_toolkit.get_tools()

agent_chain = initialize_agent(tools=tools, llm=llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

In [6]:
agent_chain.run("Hey GPT, my name is Chris, please remember it")



> Entering new AgentExecutor chain...
{
    "action": "Vector Store Memory Saver",
    "action_input": "My name is Chris"
}

Evaluating ingest: 100%|██████████| 1/1 [00:00<00:00



Observation: Answer: Remembered this fact.
Thought:{
    "action": "Final Answer",
    "action_input": "I'm sorry, but you haven't asked a question for me to answer. Is there something specific you would like to know?"
}

> Finished chain.


"I'm sorry, but you haven't asked a question for me to answer. Is there something specific you would like to know?"

In [7]:
agent_chain.run("My cat's name is Scotty")



> Entering new AgentExecutor chain...
Here is a markdown code snippet of a JSON blob with a single action to save the user's input to long-term memory using the Vector Store Memory Saver tool:

```json
{
    "action": "Vector Store Memory Saver",
    "action_input": "My cat's name is Scotty"
}
```

Evaluating ingest: 100%|██████████| 1/1 [00:00<00:00



Observation: Answer: Remembered this fact.
Thought:I apologize for the confusion earlier. Here is a markdown code snippet of a JSON blob with a single action to retrieve memories related to the user's input using the Vector Store Memory Retriever tool:

```json
{
    "action": "Vector Store Memory Retriever",
    "action_input": "What is my cat's name?"
}
```
Observation: Answer: ["My cat's name is Scotty"]
Thought:Here is a markdown code snippet of a JSON blob with a single action to directly respond to the user's question:

```json
{
    "action": "Final Answer",
    "action_input": "Your cat's name is Scotty."
}
```

> Finished chain.


"Your cat's name is Scotty."

In [14]:
agent_chain.run("Do you remember my name?")



> Entering new AgentExecutor chain...
{
    "action": "Vector Store Memory Retriever",
    "action_input": "What is my name?"
}
Observation: Answer: ["My name is Chris"]
Thought:{
    "action": "Final Answer",
    "action_input": "I'm sorry, but I don't have any information about your original question. Can you please remind me what it was?"
}

> Finished chain.


"I'm sorry, but I don't have any information about your original question. Can you please remind me what it was?"

In [13]:
agent_chain.run("What is my cat's name?")



> Entering new AgentExecutor chain...
{
    "action": "Vector Store Memory Retriever",
    "action_input": "What is my cat's name?"
}
Observation: Answer: ["My cat's name is Scotty"]
Thought:{
    "action": "Final Answer",
    "action_input": "Your cat's name is Scotty."
}

> Finished chain.


"Your cat's name is Scotty."